# MA346 Final Project: U.S. College Recommender System
// Josephine Kantawiria - MA346 Summer 2021 //

## Introduction

### Purpose of Project

Choosing a college is one of the major decisions every student needs to make as it plays a pivotal role in their career and life. However, choosing an appropriate college can be challenging. The college selection process also require a lot of searching work.

This project will focus on assisting students in choosing their college based on basic variables. It will provide the user with top 5 U.S. colleges that are deemed most suitable for the student's preference and eligibility, which the user can input. Users will be asked to input information such as: preferred location, SAT score, maximum tuition that they would accept, as well as their student status (if they are international, local out of state, or local in state). As many individuals have different needs and concerns about their college eligibility and requirements, users are also able to weigh the importance of each of the predictor variables from a scale of 1 to 10.

This college recommender system will then evaluate these different factors and display the top 5 U.S. college recommendations in the most user-friendly design. The college recommender system will then calculate a compatibility score between 0 and 1, with 1 being the most compatible and return the 5 universities with compatibility score closest to 1 to show all of the colleges that are most compatible for the user.

To check out the fully functioning Streamlit dashboard, please visit [this link](https://josephinekantawiria.herokuapp.com). More information can be found [here](https://github.com/JosephineKantawiria/MA346_FinalProject) on my GitHub repo.

## Data Pre-processing

### Load Data

For this project, we will be using two datasets - both in CSV file format. The first dataset can be accessed through the [U.S. Depatment of Education](https://data.ed.gov/dataset/college-scorecard-all-data-files-through-6-2020/resources) under the College Scorecard project resources. The College Scorecard provides data files with data about institutions as a whole and data files with data about specific fields of study within institutions. We will be using the first dataset that provide all data elements. Many data elements within the two data files are drawn directly from, or derived from, data reported to the Integrated Postsecondary Education Data System (IPEDS). The data also came with a data dictionary, which will be broken down later. Since the dataset was large, we already went ahead and hand-picked all the columns that are needed for this project.

The second dataset can be accessed from [kaggle.com](https://www.kaggle.com/joeshamen/world-university-rankings-2020). It is a dataset that comprises the ranking of the best universities of the world made by The Times Higher Education for 2020. It includes almost 1,400 universities across 92 countries, standing as the largest and most diverse university rankings ever to date. The rankings are determined based on 13 performance indicators that measure an institution’s performance across teaching, research, knowledge transfer and international outlook. The university ranking also has been independently audited by professional services firm PricewaterhouseCoopers. This information will be used to determine the rankings of the universities listed in the first dataframe, providing a more holistic view of the university's performance.  

The packages we will be using for this project are `Pandas`, `NumPy`, `Sklearn`, and `Streamlit`.

In [1]:
# Load packages
import pandas as pd
import numpy as np
from sklearn import preprocessing
import streamlit as st

# Load data
df1 = pd.read_csv('Most-Recent-Cohorts-All-Data-Elements.csv')
df2 = pd.read_csv('World_University_Rank_2020.csv')

All packages are imported and loaded. The first dataset is stored into DataFrame `df1` and the second dataset into DataFrame `df2`. After loading both data into two DataFrames, we can view the first few rows of each DataFrame.

In [2]:
# View first few rows of df1
df1.head()

,INSTNM,CITY,STABBR,LATITUDE,LONGITUDE,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT
0,Alabama A & M University,Normal,AL,34.783368,-86.568502,0.8986,957.0,9744.0,18354.0
1,University of Alabama at Birmingham,Birmingham,AL,33.505697,-86.799345,0.9211,1220.0,8568.0,19704.0
2,Amridge University,Montgomery,AL,32.362609,-86.174010,NaN,NaN,6900.0,6900.0
3,University of Alabama in Huntsville,Huntsville,AL,34.724557,-86.640449,0.8087,1314.0,10714.0,22362.0
4,Alabama State University,Montgomery,AL,32.364317,-86.295677,0.9774,972.0,11068.0,19396.0


We can derive that the first DataFrame, `df1`, has a total of 9 columns. We will do the same for the second DataFrame, `df2`, to explore the data.

In [3]:
# View first few rows of df2
df2.head()

,Rank_Char,Score_Rank,University,Country,Number_students,Numb_students_per_Staff,International_Students,Percentage_Female,Percentage_Male,Teaching,Research,Citations,Industry_Income,International_Outlook,Score_Result,Overall_Ranking
0,1,1,University of Oxford,United Kingdom,"20,664",11.2,41%,46%,54%,90.5,99.6,98.4,65.5,96.4,95.4,95.40
1,2,2,California Institute of Technology,United States,"2,240",6.4,30%,34%,66%,92.1,97.2,97.9,88.0,82.5,94.5,94.50
2,3,3,University of Cambridge,United Kingdom,"18,978",10.9,37%,47%,53%,91.4,98.7,95.8,59.3,95.0,94.4,94.40
3,4,4,Stanford University,United States,"16,135",7.3,23%,43%,57%,92.8,96.4,99.9,66.2,79.5,94.3,94.30
4,5,5,Massachusetts Institute of Technology,United States,"11,247",8.6,34%,39%,61%,90.5,92.4,99.5,86.9,89.0,93.6,93.60


We can also derive that the second DataFrame, `df2`, has a total of 16 columns. The next process is to prepare the data for analysis by removing unnecessary columns and incomplete data, as well as modify improperly formatted data.

### Data Cleaning - DataFrame 1

The first DataFrame `df1` initially was a large dataset with 2384 columns. We have went ahead and extracted only selected columns and dropping the rest from the dataset as these are the only ones we will be interacting with. The data dictionary can be found on the same website and should be located in the same ZIP file with an Excel file format. From there we decide to only keep the following columns:

- `INSTM`: Institution name
- `CITY`: City where the institution is located
- `STABBR`: State abbreviation where the institution is located
- `SAT_AVG`: Average SAT equivalent score of students admitted for all campuses
- `ADM_RATE`: Overall admission rate
- `TUITIONFEE_IN`: In-state tuition and fees
- `TUITIONFEE_OUT`: Out-of-state tuition and fees
- `LATITUDE`: Institution's latitude
- `LONGITUDE`: Institution's longitude

These were the columns that were chosen from the inital 2384 columns in the data file.

In [4]:
#View first few rows
df1.head()

,INSTNM,CITY,STABBR,LATITUDE,LONGITUDE,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT
0,Alabama A & M University,Normal,AL,34.783368,-86.568502,0.8986,957.0,9744.0,18354.0
1,University of Alabama at Birmingham,Birmingham,AL,33.505697,-86.799345,0.9211,1220.0,8568.0,19704.0
2,Amridge University,Montgomery,AL,32.362609,-86.174010,NaN,NaN,6900.0,6900.0
3,University of Alabama in Huntsville,Huntsville,AL,34.724557,-86.640449,0.8087,1314.0,10714.0,22362.0
4,Alabama State University,Montgomery,AL,32.364317,-86.295677,0.9774,972.0,11068.0,19396.0


Now `df1` only contains the 9 columns we have defined and only contain data that is relevant to the U.S. college recommender system project.

### Data Cleaning - DataFrame 2

We will be taking similar steps for `df2`. The data dictionary for `df2` can be found on the same link as the file download. Since we already have most of the information for the universities broken down in a much more detailed manner in `df1`, we will simply be using this DataFrame for the university rankings. Hence, the columns that are most relevant for this Data are the following:

- `Rank_Char`: Ranking according to The Times Higher Education
- `University`: Name of the university
- `Country`: Country of the university

Using these three columns, we will be able to further filter both the rows and the columns within `df2` to only contain data on univesities that are located within United States by filtering the `Country` column. In other words, we can drop all data of universities that are not located in the United States. Once filtered by country, we actually no longer need to see the `Country` column as all the columns will show "United States" and becomes irrelevant to the dataset. We can then drop the `Country` column and only keep the remaining two.

In [5]:
# Filter rows and columns
df2 = df2.loc[df2["Country"] == "United States"][["Rank_Char", "University"]]

In [6]:
#View first few rows
df2.head()

,Rank_Char,University
1,2,California Institute of Technology
3,4,Stanford University
4,5,Massachusetts Institute of Technology
5,6,Princeton University
6,7,Harvard University


Now, `df2` only contains universities that are located in the United States and only display the two columns that are needed, the `Rank_Char` and `University` column. However, a new problem arises as we see that the `Rank_Char` column is not an integer column and instead contains string values. We can view the problem by analyzing all the unique value in the `Rank_Char` column.

In [7]:
# View all unique values
df2['Rank_Char'].unique()

array(['2', '4', '5', '6', '7', '8', '9', '11', '12', '13', '16', '17',
       '19', '20', '21', '22', '26', '27', '29', '31', '38', '48', '51',
       '52', '53', '54', '55', '57', '61', '62', '70', '78', '79', '80',
       '84', '88', '91', '94', '96', '102', '104', '105', '107', '113',
       '116', '119', '124', '134', '139', '155', '157', '168', '172',
       '173', '175', '178', '179', '198', '201�250', '251�300', '301�350',
       '351�400', '401�500', '501�600', '601�800', '801�1000', '1001+'],
      dtype=object)

After further investigation, it is understood that some values are not integers because some values within the `Rank_Char` column are written as a range. In the case of a range, we will take the middle value, for example the university ranking will be 275 for universities in the ranking range of 251 to 300. The rankings also end at 1000, so all universities above that range is communicated as 1001+. For this, we will set a uniform value of 1001 for all universities above the range of 1001..

In [8]:
# Replace values
df2["Rank_Char"].replace(
    {
        "201�250": 225,
        "251�300": 275,
        "301�350": 325,
        "351�400": 375,
        "401�500": 425,
        "501�600": 525,
        "601�800": 700,
        "801�1000": 900,
        "1001+": 1001,
    },
    inplace=True,
)

# Convert column type
df2['Rank_Char'] = pd.to_numeric(df2['Rank_Char'])

In [9]:
# Check all unique values
df2['Rank_Char'].unique()

array([   2,    4,    5,    6,    7,    8,    9,   11,   12,   13,   16,
         17,   19,   20,   21,   22,   26,   27,   29,   31,   38,   48,
         51,   52,   53,   54,   55,   57,   61,   62,   70,   78,   79,
         80,   84,   88,   91,   94,   96,  102,  104,  105,  107,  113,
        116,  119,  124,  134,  139,  155,  157,  168,  172,  173,  175,
        178,  179,  198,  225,  275,  325,  375,  425,  525,  700,  900,
       1001])

We have succesfully replaced all value ranges into integers using the `replace()` function and the dictionary, as well as converting the rest of the values to numeric values. Now, we can see that the unique values no longer include any strings. Finally, `df2` is now ready to use. 

### Data Merging

We are now ready to combining the two DataFrames into a single DataFrame, which we will call `df`. The two DataFrames will be merged on the university name, the column is called `INSTNM` on `df1` and `University` on `df2`. We will only include universities that have a rank.

In [10]:
# Merge DataFrames
df = df1.merge(df2, how='inner', left_on='INSTNM', right_on='University')
del df['University']

In [11]:
# View first few rows
df.head()

,INSTNM,CITY,STABBR,LATITUDE,LONGITUDE,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,Rank_Char
0,University of Alabama at Birmingham,Birmingham,AL,33.505697,-86.799345,0.9211,1220.0,8568.0,19704.0,172
1,University of Alabama in Huntsville,Huntsville,AL,34.724557,-86.640449,0.8087,1314.0,10714.0,22362.0,525
2,Auburn University,Auburn,AL,32.599378,-85.488258,0.7543,1289.0,11276.0,30524.0,700
3,University of Alaska Fairbanks,Fairbanks,AK,64.857560,-147.823146,0.7611,1128.0,6657.0,20218.0,375
4,University of Arizona,Tucson,AZ,32.232672,-110.950815,0.8442,1198.0,12467.0,36366.0,104


Now the two DataFrames are merged into one called `df`. It originally contained 11 columns but `University` was dropped as it contains duplicate values with column `INSTNM`. The DataFrame now contains 10 columns in which we will use to build a U.S. College Recommender System.

## Building the Recommender System

### User Input

Users will be able to interact with the recommender system to input their data. We will be using `Streamlit` dashboard for interactive widgets. Data that users need to input are:

- `student_status`: Refers to the way the user was enrolled in (International, Local In-State, or Local Out-of-State)
- `sat_score`: User's SAT score in the 1600 range
- `max_tuition`: The maximum tuition that the user would accept
- `pref_locations`: User's preferred college locations
- `scale_tuition`: The importance of tuition match in user's college selection from a scale of 1(lowest) to 10(highest)
- `scale_sat`: The importance of SAT match in user's college selection from a scale of 1(lowest) to 10(highest)
- `scale_admission`: The importance of admission rate in user's college selection from a scale of 1(lowest) to 10(highest)
- `scale_rank`: The importance of college ranking in user's college selection from a scale of 1(lowest) to 10(highest)

All these inputs are interactive and can be displayed in different types of widgets on `Streamlit`.

In [12]:
# Function for user input
def user_input1():
    # Define student status
    student_status = st.selectbox('Select your student status: ', ['Choose an option', 'International', 'Local In-State', 'Local Out-of-State'])
    
    # Define SAT score
    sat_score = st.number_input('Input your SAT score: ', min_value=0, max_value=1600)

    # Define maximum tuition
    min_value = int(min(df['TUITIONFEE_IN'].min(), df['TUITIONFEE_OUT'].min()))
    max_value = int(max(df['TUITIONFEE_IN'].max(), df['TUITIONFEE_OUT'].max()))
    max_tuition = st.slider('Select the maximum tuition you would accept: ', min_value= min_value, max_value= max_value, value = 10000)

    # Define preffered location - Multiselect widget
    locations = np.append(df['STABBR'].unique(), "All")
    pref_locations = st.multiselect("Select your preferred location(s): ", locations, help="Data will be filtered according to the counties selected. Select \"All\" or none to view data from all counties.")
    
    # If preferred location is All or None
    if ('All' in pref_locations) or (len(pref_locations) == 0):
        pref_locations = df['STABBR'].unique()

    return student_status, sat_score, max_tuition, pref_locations

In [13]:
def user_input2():
    # Define variable importance from 1 to 10
    scale_tuition = st.slider("From a scale of 1 (lowest) to 10 (highest), rate the importance of a low tuition rate in your college search: ", min_value=1, max_value=10, value=5)
    scale_sat = st.slider("From a scale of 1 (lowest) to 10 (highest), rate the importance of an SAT match in your college search: ", min_value=1, max_value=10, value=5)
    scale_admission = st.slider("From a scale of 1 (lowest) to 10 (highest), rate the importance of a high admission rate in your college search: ", min_value=1, max_value=10, value=5)
    scale_rank = st.slider("From a scale of 1 (lowest) to 10 (highest), rate the importance of a high college ranking in your college search: ", min_value=1, max_value=10, value=5)

    return scale_tuition, scale_sat, scale_admission, scale_rank

The function above calls for users to input their data through the use of various interactive widgets. The user input is divided into two sections for aesthetic purposes: `user_input1()` and `user_input2()`.

In the first function `user_input1()`, the `student_status` is a select box that allows for the users to choose out of the 3 available options. This will allow the users to just select one out of the three instead inputting their own text. The default is an added option of 'Choose your option' that holds no value but a placeholder for the default index. When users do not select out of the 3 actual available options, they are not able to derive their data

The `sat_score` is a number input widget with a minimum value of 0 and maximum value of 1600 as it is the range of the SAT score. 

The `max_tuition` utilizes the slider widget that allows user to select a tuition that is within the minimum and maximum range. It takes a numerical data as range input with minimum be the smallest value in the `TUITIONFEE_IN` or `TUITIONFEE_OUT` column (whichever is lower) and the maximum being the highest value between the same two columns (whichever is highest). 

The `pref_locations` is a multiselect box which allows users to select multiple locations as their preferred college location. The options are all unique values from the `STABBR` column, which contains 44 unique state abbreviations, and an option to select 'All'. If a user selects 'All' in their preferred location or does not select any, it will not filter the data by any specific value and instead takes all state abbreviations that are unique in the DataFrame as its value.

In the next function `user_input2()`,  the `scale_tuition`, `scale_sat`, `scale_admission`, and `scale_rank` all uses a slider option with range between minimum of 1, lowest importance, to maximum of 10, highest importance. User's are able to determine what their top priorities are within their college selection process.

The function will then return all of the variables listed above and the first four (`student_status`, `sat_score`, `max_tuition`, and `pref_locations`) will be used to filter the DataFrame.

### Filter DataFrame by User Input

We would want to filter the DataFrame for all universities that are located in the user's preferred location(s), accept applicants with equivalent or lower average SAT scores with the user's SAT score, and charge a tuition rate that is lower than or equivalent to their maximum tuition rate.

In [14]:
def decide_tuition_category (student_status):
    tuition_category = ''
    # Tuition based on student status
    if student_status == 'International' or student_status == 'Local Out-of-State':
        tuition_category = 'TUITIONFEE_OUT' # Use out-of-state tuition
    elif student_status == 'Local In-State':
        tuition_category = 'TUITIONFEE_IN' # Use in-state tuition
    
    return tuition_category

The function will filter for the tuition rate that is below or equivalent to the maximum tuition the user will accept, it must be taken into account that we shold use out-of-state tuition for international or local out-of-state applicants and in-state tuition for local in-state applicants, as these prices adjust to the student status. Hence, we create a function that decides on the `tuition_category`.

In [15]:
def filter_data (sat_score, max_tuition, pref_locations, tuition_category):
    # Filter DataFrame
    mask = (df['STABBR'].isin(pref_locations)) & (df['SAT_AVG'] <= sat_score) & (df[tuition_category] <= max_tuition)
    filtered_df = df.loc[mask]
    
    return filtered_df

The function takes the values returned by the `user_input` function above and filters the data accordingly. Lastly, we filtered for universities that have an average SAT score that is lower than or equivalent to the user's SAT score to ensure that the SAT scores are aligned. The function then returns a filtered DataFrame, `filtered_df` and the variable `tuition_category` as it will be used to determine which tuition rate to use in future functions.

### Normalize Columns

Now that we have a function to filter the DataFrame, we also need to create a function to normalize certain columns within that DataFrame so that we can generate a score. We see that our data are all measured in different scales. The goal of normalization is to make every datapoint have the same scale so each feature is equally important. This is important because otherwise the feature with the larger scale will completely dominate the rest of the data.

We will be doing a min-max normalization as it is one of the most common ways to normalize data. Min-max normalization performs a linear transformation on the original data. How it works is that for every feature, the minimum value of that feature gets transformed into a 0, the maximum value gets transformed into a 1, and every other value gets transformed into a decimal between 0 and 1. The formula can be written as follow:

> $v_i = (v_i - min_A) / (max_A - min_A)$

where $v_i$ is the value, $min_A$ is minimum value in the attribute, and $max_A$ is the maximum value in the attribute. We will be using `sklearn.preprocessing.MinMaxScaler` to transform features by scaling each feature to a given range.

In [16]:
# Function to normalize column by min-max scaling
def normalize (column):
    col_array = filtered_df[column].to_numpy()
    min_max_scaler = preprocessing.MinMaxScaler()
    col_scaled = min_max_scaler.fit_transform(col_array.reshape(-1, 1))
    return col_scaled.flatten()

The `normalize` function will take into a column and conver it to a numpy array. Using the `MinMaxScaler()` and `fit_transform` the column into the scaler to fit to data, then transform it to a transformed version of X. We also needed to reshape the column to a way that the colun array has only 1 column to fit into the scaler. This allows us to get an array of scaled values. We then need to use the `flatten()` function is used to get a copy of an given array collapsed into one dimension, which will later be used in the actual recommendation score calculation.

We will normalize the following columns: `SAT_AVG`, `TUITIONFEE_OUT` or `TUITIONFEE_IN` (depending on the status of the student), and `Rank_Char`.

### Calculating Weight of Importance

When asking for user input, we also asked for the importance of each variable to their college search. It was given to them as a scale of 1 to 10, where 1 is lowest and 10 is highest. To use this number, we need to convert it into a ratio to assess how much each variable actually weighs.

In [17]:
# Function to calculate weight of importance of variables
def calculate_weight(variable):
    return variable / sum([scale_tuition, scale_sat, scale_admission, scale_rank])

We take the user_input for each variable and divide the number by the sum of the scale. For example, the user inputs 10, 10, 10, 10 for all respective variables, this means that each variable weighs 10/(10 + 10 + 10 + 10) or 0.25 as each of them has the same importance. If the user inputs 10, 9, 7, 5 then variable one would weight 10/(10 + 9 + 7 + 5) or 0.32.

### Calculate University Compatibility Scores

With everything already set up, we can now calculate the compatibility scores, which measures how much of a match each university is to the user's data and preference. To calculate each individual score, the following formula will be used:

>score = weight_tuition \* (1 - tuition) +  weight_sat \* (1 - sat average) + weight_admission \* admission + weight_rank \* (1 - rank)

We will be using the scaled DataFrame to generate these scores. First, it is understood that the lower the tuition, the higher the compatibility score will be, which proposes an inverse relationship. The lower the tuition rate is, the more attractive for most people and hence more compatible, this explains why we calculate tuition as 1 - tuition. If the user highly prioritize tuition match in their college search, then the `weight_tuition` will be the highest compared to others. The higher the weight for the tuition then the results will return more colleges with lower tuition.

Same for SAT score, we would want the lowest average SAT score as an average SAT score that is lower than the user's SAT score will increase the chances of the user being admitted. Hence, it has an inverse relationship with compatibility. Similarly, `weight_sat` depends on the user and how safe they want to be in determining their SAT match. The higher the weight for the SAT then the results will return more colleges with lower average SAT.

Admission rates however have a positive relationship with compatibility score as a higher admission rate increases chance of users being admitted to a college. The `weight_admission` depends on how safe they want to be in terms of admission match. The higher the weight for the admission then the results will return more colleges with higher admission rate.

Rankings also have an inverse relationship with compatibility score as a smaller ranking indicates a better college. If the user wants to be in a college with a high ranking (i.e. lower number) will have a higher `weight_rank`.  The higher the weight for the rank then the results will return more colleges with higher rankings (i.e. lower number).

In [18]:
# Function to pull top scores
def calculate_scores ():
    
    # Define empty dictionary
    scores = {}

    # Convert DataFrame to list
    university_names = filtered_df['INSTNM'].tolist()
    sat_avgs = normalize('SAT_AVG')
    admissions = filtered_df['ADM_RATE'].tolist()
    tuitions = normalize(tuition_category)
    ranks = normalize('Rank_Char')

    # Calculate weight of each variable
    weight_tuition = calculate_weight(scale_tuition)
    weight_sat = calculate_weight(scale_sat)
    weight_admission = calculate_weight(scale_admission)
    weight_rank = calculate_weight(scale_rank)

    # Calculate scores
    for i in range(len(filtered_df)):
        scores[university_names[i]] = weight_tuition * (1 - tuitions[i]) +  weight_sat * (1 - sat_avgs[i]) + weight_admission * admissions[i] + weight_rank * (1 - ranks[i])

    return scores

Once we have all the scores calculated for each university stored in a dictionary called `scores`, we can sort the values in descending order and pull the top 5. These universities are the most compatible to the user's data and preferences.

In [19]:
def top_scores (scores):
    # Sort universities by top 5 scores
    top5 = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:5]
    
    # Only extract university name into list
    top5 = [name for x in top5 for name in x[::2]]

    return top5

The `top5` list contains the college name and the compatibility score, we only need the college name in a list sorted by the score, hence we can just take the name into the list. We will then be able to locate the college names in the original DataFrame to pull the original data for the users to view and explore.

## Streamlit Dashboard

### Streamlit Features

Streamlit is a great new tool that allows users to quickly build highly interactive web applications around their data without much knowledge on web development. The use of Streamlit for this project enhances the user experience as they are able to access the data via a user-friendly web page.

Although Streamlit is helpful in displaying data, charts, and utilzing interactive widgets, there are some features that Streamlit is still in the process of developing. One of them is a multi-page feature. They have not yet launched a feature for a multi-page app so many in the community found a work around by using select boxes or radio buttons on the sidebar and pages that are embedded into functions to mimick the use of a navigation bar.

We will be doing this for our dashboard to give the look that the Streamlit dashboard has multiple pages when actually it is the content that is changing. We will have a total of 3 pages accessible from the navigation bar: Main, Explore the Database, and the College Recommender System.

### Page One - Main Page

The main page serves as welcome page that contains information regarding the purpose of the project, information about the college recommender system, and information about the data. It also features a welcome banner. The main page is stored in a function called `page_one()`.

In [20]:
def page_one():
    # Title and name
    st.title('MA346 Summer 2021 - Final Project')
    st.write("by: Josephine Kantawiria - MA346 Summer 2021")

    # Welcome banner image
    st.image('WelcomeBanner.png')

    # Purpose of Project
    st.subheader("Purpose of Project")
    st.markdown(''' Choosing a college is one of the major decisions every student needs to make as
    it plays a pivotal role in their career and life. However, choosing an appropriate college
    can be challenging. The college selection process also require a lot of searching work. This project
    will focus on assisting students in choosing their college based on basic variables. It will provide
    the user with top 5 U.S. colleges that are deemed most suitable for the student's preference and
    eligibility, which the user can input.''')
    st.write()

    # Information about the College Recommender System
    st.subheader("About the College Recommender System")
    st.markdown('''Users will be asked to input information such as: preferred location,
    SAT score, maximum tuition that they would accept, as well as their student status
    (if they are international, local out of state, or local in state). As many individuals
    have different needs and concerns about their college eligibility and requirements, users
    are also able to weigh the importance of each of the predictor variables from a scale of 1 to 10.
    This college recommender system will then evaluate these different factors and display the top 5
    U.S. college recommendations in the most user-friendly design.''')
    st.write()

    # Information about the Data
    st.subheader("Information about the Data")
    st.markdown('''For this project, we will be using two datasets - both in CSV file format.
    The first dataset can be accessed through the [U.S. Depatment of Education]
    (https://data.ed.gov/dataset/college-scorecard-all-data-files-through-6-2020/resources) under
    the College Scorecard project resources. The College Scorecard provides data files with data
    about institutions as a whole and data files with data about specific fields of study within
    institutions. We will be using the first dataset that provide all data elements. Many data
    elements within the two data files are drawn directly from, or derived from, data reported
    to the Integrated Postsecondary Education Data System (IPEDS). The data also came with a data
    dictionary, which will be broken down later.''')
    st.markdown('''The second dataset can be accessed from [kaggle.com]
    (https://www.kaggle.com/joeshamen/world-university-rankings-2020). It is a dataset that comprises
    the ranking of the best universities of the world made by The Times Higher Education for 2020.
    It includes almost 1,400 universities across 92 countries, standing as the largest and most
    diverse university rankings ever to date. The rankings are determined based on 13 performance
    indicators that measure an institution’s performance across teaching, research, knowledge
    transfer and international outlook. The university ranking also has been independently audited
    by professional services firm PricewaterhouseCoopers. This information will be used to determine
    the rankings of the universities listed in the first dataframe, providing a more holistic view
    of the university's performance.''')

    # More Information
    st.subheader("More Information on Project")
    st.markdown('''More information can be found [here](https://github.com/JosephineKantawiria/MA346-FinalProject) on my GitHub repo.''')

### Page Two - Explore the Data

The second page consist of the DataFrame and a map. In this page, users are able to scroll through all the data and view their locations on the map. This page is stored in a function called `page_two()`.

In [21]:
def page_two():
    # Title
    st.title('Explore the Data')

    # DataFrame to display
    df_display = df[['INSTNM', 'CITY', 'Rank_Char', 'SAT_AVG', 'ADM_RATE', 'TUITIONFEE_IN', 'TUITIONFEE_OUT']]

    # Rename columns
    df_display.columns = ['College Name', 'City', 'Rank', 'Average SAT Score', 'Admission Rate', 'Tuition Rate (In-State)', 'Tuition Rate (Out-of-State)']

    # DataFrame for map
    df_map = df[['LATITUDE', 'LONGITUDE']]
    df_map.columns = ['lat', 'lon']

    # Display DataFrame
    st.dataframe(df_display.sort_values(by=['Rank']))

    # Display map
    st.map(df_map)

### Page Three - College Recommender System

The college recommender system is located in its own page. It is actually divided into two parts, `page_three_a` and `page_three_b`. It is divided into two for aesthetic reasons as `page_three_b` will only appear after clicking the 'Next' button at the end of `page_three_a`.

In `page_three_a` the page is also divided into two column with a buffer column in the middle used as padding. This is done to allow the two user input functions to be placed next to each other and takes up less space. However, as a result of splitting the page into two parts, we need to make the variables within the page functions as a global function so that it is able to be accessed from outside of the function. Some functions in `page_three_b` uses these variables in its calculations.

At the end of  `page_three_a`, there is a button that says 'Next'. This button is what will activate the `page_three_b` function. However, after the user finishes filling out all of their data and there are no matches within `filtered_df`, it will instead return an error message.

In [22]:
def page_three_a():
   # Title
   st.title('College Recommender System')
   
   # Horizontal line
   st.markdown('''---''')

   # Create first row
   row1 = st.beta_container()

   # Split columns
   col1, buff, col2 = st.beta_columns([10, 1, 10])
   
   # Row 1, Col 1
   with row1:
      with col1:
         # Call first user_input function
         student_status, sat_score, max_tuition, pref_locations = user_input1()

   # Row 1, Col 2
   with row1:
      with col2:
         # Global variable
         global scale_tuition, scale_sat, scale_admission, scale_rank
         # Call second user input function
         scale_tuition, scale_sat, scale_admission, scale_rank = user_input2()
         
         # Do not proceed if information is incomplete
         if student_status != 'Choose an option':
            # Global variable
            global tuition_category
            # Call function to decide tuition category based on student status
            tuition_category = decide_tuition_category(student_status)

            # Global variable
            global filtered_df
            # Call function to filter the data
            filtered_df = filter_data (sat_score, max_tuition, pref_locations, tuition_category)

   # Horizontal line
   st.markdown('''---''')

   # Create second row
   row2 = st.beta_container()

   # Row 2
   with row2:
      # Create button
      if st.button('Next'):
         # If there is no data
         if student_status == 'Choose an option' or len(filtered_df) == 0:
            # Error message
            st.error('Sorry, no university matches for you currently.')
         # If there is data
         else:
            # Activate next page
            page_three_b()

If there are matches, the next page is activated and will continue to calculate the scores for all the universities in `filtered_df` and return the top 5 universities based on highest compatibility score. The information will be shown in a DataFrame with renamed column headers and an index that starts from 1. It will also generate a map below it to point to where the institution is located. The map is interactive and users can zoom in and out to view the points.

In [23]:
def page_three_b():
    # Call function to calculate scores
    scores = calculate_scores()

    # Call function to pull top 5 college names based on highest scores
    top5 = top_scores(scores)

    # DataFrame to display
    top5_df_display = df.loc[df['INSTNM'].isin(top5),['INSTNM', 'CITY', 'Rank_Char', 'SAT_AVG', 'ADM_RATE', tuition_category]]
    
    # Rename columns
    top5_df_display.columns = ['College Name', 'City', 'Rank', 'Average SAT Score', 'Admission Rate', 'Tuition Rate']
    
    # Reset index to start at 1
    top5_df_display.index = np.arange(1, len(top5_df_display) + 1)

    # DataFrame for map
    top5_df_map = df.loc[df['INSTNM'].isin(top5),['LATITUDE', 'LONGITUDE']]
    top5_df_map.columns = ['lat', 'lon']

    # Display DataFrame
    st.dataframe(top5_df_display)

    # Display Map
    st.map(top5_df_map)

### Putting All the Pages Together

To put everything together, we need to set up a radio button that is locted on the sidebar as a way to mimick the navigation bar look. The radio button has three buttons that the user can choose from as mentioned above, which coresponds to the page chosen. The default is at the main page.

In [24]:
page = st.sidebar.radio('Navigation Bar', ('Main', 'Explore the Data', 'College Recommender System'))

if page == 'Main':
    page_one()
if page == 'Explore the Data':
    page_two()
if page == 'College Recommender System':
    page_three_a()

2021-06-29 17:59:30.324 
  command:

    streamlit run /shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py [ARGUMENTS]


## Conclusion

Now we have a completed college recommender system that returns the top 5 universities based on the user's data. The complete rundown is as follow: The user is first asked to select their student status which will determine which tuition rate category to be used, in-state or out-of-state. The user will then be asked to input their SAT score in the 1600 range. Followed by that, the user can use the slider to select the maximum tuition they would accept, the minimum tuition is the lowest tuition derived from the DataFrame and the maximum is the highest tuition derived from the DataFrame. The user is then asked to select their preferred location(s) using a multi-select box widget. The user has the option to choose 'All' or select none to filter for all locations, or choose as many locations as they want from the options of states. After that the user will be prompted to select the level of importance from a scale of 1 (lowest) to 10(highest) for tuition rate, SAT match, admission rate, and college ranking. Once completed, the user can press next.

The college recommender system will then evaluate all these factors and insert it into the given formula to calculate a compatibility score between 0 to 1, with 1 being the most compatible.

### More Information

To check out the fully functioning Streamlit dashboard, please visit [this link](https://josephinekantawiria.herokuapp.com). More information can be found [here](https://github.com/JosephineKantawiria/MA346_FinalProject) on my GitHub repo.

In [25]:
!jupyter nbconvert --to script FinalProject.ipynb

[NbConvertApp] Converting notebook FinalProject.ipynb to script
[NbConvertApp] Writing 12473 bytes to FinalProject.txt


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=11720eb9-b5e7-4041-bb0b-ca4065cfb8e6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>